In [1]:
import numpy as np
import pandas as pd

In [2]:
dataFilter = [['FullBiasNanos','FullBiasNanos ~= 0'],['ConstellationType','ConstellationType==1'],['State','bitand(State,2^0) & bitand(State,2^3)']]
gnssAnalyze = {
    "GnssClockErrors":'GnssClock Errors.' ,
    "GnssMeasurementErrors":'GnssMeasurement Errors.',
    "ApiPassFail":'' 
}

In [3]:
prFileName = 'in01_G.txt'
dirName = '/home/lyt/gnss_wifi/gnss/indoorGPS/'
outputname = 'outplot/1out_in'

In [4]:
f = open(dirName + prFileName)
datastring = f.read()
f = open(dirName + prFileName)
data = f.readlines()
f.close()
iDigits = datastring.find('Version')
version = datastring[iDigits+9:iDigits+17]
iDigits = datastring.find('Platform')
platform = datastring[iDigits+10:iDigits+15]

In [5]:
version, platform

('v2.0.0.1', '8.0.0')

In [6]:
rawlist = []
for i in range(len(data)):
    if data[i].find('Raw') != -1 and data[i].find('# Raw')==-1 :
        rawlist.append(i)
#print(rawlist) #12,22  37,46

In [7]:

head = data[5].split(',')[1:]
head[-1] = head[-1][:head[-1].find('\n')]

df2 = pd.DataFrame(columns=head)
for j in range(len(rawlist)):
    i = rawlist[j]
    if j == 0:
        time = 0
        df = pd.DataFrame(columns=head)
    ldata_f = []
    ldata = data[i].split(',')[1:]
    ldata[-1] = ldata[-1][:ldata[-1].find('\n')]
    for m in ldata:
        if m != '':
            ldata_f.append(float(m))
        else:
            ldata_f.append(m)
    #print(j)
    ldf = pd.DataFrame([ldata_f],columns=head)
    
    df = df.append(ldf,ignore_index=True)
    
    if i == rawlist[-1]:
        #df = df.sort_values(by=['Cn0DbHz'],ascending=False)
        #df = df.drop_duplicates('Svid')
        #df = df.reset_index()
        df = pd.concat([df],keys=['t'+ str(time)])
        df = df.assign(allRxMills = (df.TimeNanos - df.FullBiasNanos)*1e-6)
        df_g = pd.concat([df_g,df])
        
    elif rawlist[j+1] -i >1:
        #df = df.sort_values(by=['Cn0DbHz'],ascending=False)
        #df = df.drop_duplicates('Svid')
        #df = df.reset_index()
        df = pd.concat([df],keys=['t'+ str(time)])
        df = df.assign(allRxMills = (df.TimeNanos - df.FullBiasNanos)*1e-6)
        if time == 0:
            df_g = pd.concat([df])
        else:
            df_g = pd.concat([df_g,df])
        time += 1
        df = pd.DataFrame(columns=head)
    
    


In [241]:
df_g.loc['t0'][["TimeNanos",'FullBiasNanos',"Svid","ReceivedSvTimeNanos",'PseudorangeRateMetersPerSecond','AccumulatedDeltaRangeMeters','CarrierPhase']]

,TimeNanos,FullBiasNanos,Svid,ReceivedSvTimeNanos,PseudorangeRateMetersPerSecond,AccumulatedDeltaRangeMeters,CarrierPhase
0,3.468015e+12,-1.278314e+18,6.0,3.982837e+13,691.213928,0.0,
1,3.468015e+12,-1.278314e+18,1.0,3.982836e+13,-722.251892,0.0,
2,3.468015e+12,-1.278314e+18,21.0,3.982837e+13,99.809357,0.0,
3,3.468015e+12,-1.278314e+18,7.0,3.982838e+13,179.905838,0.0,
4,3.468015e+12,-1.278314e+18,8.0,3.982837e+13,-481.582062,0.0,
5,3.468015e+12,-1.278314e+18,2.0,3.746464e+14,-219.770157,0.0,
6,3.468015e+12,-1.278314e+18,5.0,3.746464e+14,-43.929295,0.0,
7,3.468015e+12,-1.278314e+18,6.0,3.746464e+14,109.837509,0.0,
8,3.468015e+12,-1.278314e+18,12.0,3.746464e+14,256.517944,0.0,
9,3.468015e+12,-1.278314e+18,13.0,3.746464e+14,-648.501831,0.0,


In [9]:
df_g.keys()

Index(['ElapsedRealtimeMillis', 'TimeNanos', 'LeapSecond',
       'TimeUncertaintyNanos', 'FullBiasNanos', 'BiasNanos',
       'BiasUncertaintyNanos', 'DriftNanosPerSecond',
       'DriftUncertaintyNanosPerSecond', 'HardwareClockDiscontinuityCount',
       'Svid', 'TimeOffsetNanos', 'State', 'ReceivedSvTimeNanos',
       'ReceivedSvTimeUncertaintyNanos', 'Cn0DbHz',
       'PseudorangeRateMetersPerSecond',
       'PseudorangeRateUncertaintyMetersPerSecond',
       'AccumulatedDeltaRangeState', 'AccumulatedDeltaRangeMeters',
       'AccumulatedDeltaRangeUncertaintyMeters', 'CarrierFrequencyHz',
       'CarrierCycles', 'CarrierPhase', 'CarrierPhaseUncertainty',
       'MultipathIndicator', 'SnrInDb', 'ConstellationType', 'AgcDb',
       'CarrierFrequencyHz', 'allRxMills'],
      dtype='object')

In [10]:
#df_g['Cn0DbHz']['t'+str(0)].mean()

In [247]:
def gps2utc(allRxMills):
    svtest = 1e-3*np.array(allRxMills)
    HOURSEC = 3600
    MINSEC = 60
    monthDays=[31,28,31,30,31,30,31,31,30,31,30,31]
    DAYSEC = 86400
    days = np.array(np.floor(svtest/DAYSEC)+6)
    years = np.zeros([days.size])+1980
    leap = np.ones([days.size])
    time = np.zeros([days.size,6])
    while((days > leap+365).any()):
        I = np.where(days > (leap+365),1,0)
        days = days - I * (leap + 365)
        years =years + I;
        leap = leap*(1-I) + np.where(years%4 == 0,1,0)
    time[:,0] = years
    
    for i in range(days.size):
        month = 1
        if (years[i]%4 == 0):  #This works from 1901 till 2099
            monthDays[1]=29 #Make February have 29 days in a leap year
        else:
            monthDays[1]=28
        while (days[i] > monthDays[month]):
            days[i] = days[i]-monthDays[month-1]
            month = month+1
        time[i,1] = month
    
    time[:,2] = days
    sinceMidnightSeconds = svtest % DAYSEC
    time[:,3] = np.floor(sinceMidnightSeconds/HOURSEC)
    lastHourSeconds = sinceMidnightSeconds % HOURSEC
    time[:,4] = np.floor(lastHourSeconds/MINSEC)
    time[:,5] = lastHourSeconds%MINSEC
    return time

In [250]:
utcTime = gps2utc(df_g.allRxMills.t0[0])
utcTime

array([[2020.        ,    7.        ,    9.        ,    8.        ,
           4.        ,    6.44112062]])

In [264]:
yearNumber4Digit = utcTime[0][0];
yearNumber2Digit = utcTime[0][0]%100;

def JulianDay(time):
    y = time[0]
    m = time[1]
    d = time[2]
    h = time[3] + time[4]/60 + time[5]/3600;
    if m<=2: #index into months <=2
        m = m + 12
        y = y - 1
    return np.floor(365.25*y) + np.floor(30.6001*(m+1)) -15
jDay = JulianDay([utcTime[0][0],utcTime[0][1],utcTime[0][2],0,0,0])
jDayJan1 = JulianDay([utcTime[0][0],1,1,0,0,0])
dayNumber = jDay - jDayJan1 + 1

In [276]:
hourlyZFile = 'hour{:03d}0.{:02d}n.Z'.format(112,10)

In [280]:
ephFilename = hourlyZFile.split('.Z')[0]

In [304]:
import os 
if not os.path.isdir("../gnss/GPSephermise"):
    os.makedirs("../gnss/GPSephermise")
else:
    if os.path.isfile("../gnss/GPSephermise/a.txt"):
        allGpsEph = 

In [315]:
ftpSiteName = 'cddis.gsfc.nasa.gov'
hourlyDir = '/gnss/data/hourly/%4d/%03d/' + str(yearNumber4Digit) + str(dayNumber)
try:
    print(z)
except:
    print('\nAutomatic ftp download failed.\n')
    print('Please go to this ftp, ftp://cddis.gsfc.nasa.gov/ \n')
    print('and get this file:{:s}{:s} \n'.format(hourlyDir,hourlyZFile))
    print('Extract contents to the directory with your pseudorange log file:\n')
    print('{:s}\n'.format(dirName))
    print('To unzip the *.Z file, see http://www.gzip.org/\n')
    print('Then run this function again, it will read from that directory\n')


Automatic ftp download failed.

Please go to this ftp, ftp://cddis.gsfc.nasa.gov/ 

and get this file:/gnss/data/hourly/%4d/%03d/2020.0183.0hour1120.10n.Z 

Extract contents to the directory with your pseudorange log file:

/home/lyt/gnss_wifi/gnss/indoorGPS/

To unzip the *.Z file, see http://www.gzip.org/

Then run this function again, it will read from that directory

